# DocGemma on Google Colab

Agentic medical AI assistant powered by MedGemma, with autonomous tool calling for clinical decision support.

This notebook deploys the full DocGemma stack (vLLM + backend + frontend) and exposes it via a public URL.

**Requirements:**
- GPU runtime (Runtime > Change runtime type > T4 or A100)
- [HuggingFace token](https://huggingface.co/settings/tokens) with access to [MedGemma](https://huggingface.co/google/medgemma-27b-it)

| GPU | VRAM | Model |
|-----|------|-------|
| T4 | 16 GB | MedGemma 1.5 4B |
| A100 (40 GB) | 40 GB | MedGemma 1.5 4B |
| A100 (80 GB) | 80 GB | MedGemma 27B |

**Repos:** [docgemma-app](https://github.com/galinilin/docgemma-app) | [docgemma-connect](https://github.com/galinilin/docgemma-connect) | [docgemma-frontend](https://github.com/galinilin/docgemma-frontend)

## 1. GPU Check & Configuration

In [ ]:
import subprocess, os

# Check GPU
result = subprocess.run(["nvidia-smi", "--query-gpu=name,memory.total", "--format=csv,noheader,nounits"],
                        capture_output=True, text=True)
if result.returncode != 0:
    raise RuntimeError("No GPU detected. Go to Runtime > Change runtime type > select T4 or A100.")

gpu_name, gpu_mem = result.stdout.strip().split(", ")
gpu_mem = int(gpu_mem)
print(f"GPU: {gpu_name} ({gpu_mem} MB VRAM)")

# Auto-select model based on VRAM
if gpu_mem >= 48000:
    MODEL = "google/medgemma-27b-it"
else:
    MODEL = "google/medgemma-1.5-4b-it"

print(f"Model: {MODEL}")

# Ports
VLLM_PORT = 8000
APP_PORT = 8080
WORKDIR = "/content/docgemma"
os.makedirs(WORKDIR, exist_ok=True)

## 2. HuggingFace Token

Enter your HuggingFace token. You need access to the [MedGemma model](https://huggingface.co/google/medgemma-27b-it) (click "Request access" on the model page).

In [ ]:
import getpass

# Try Colab secrets first, fall back to manual input
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get("HF_TOKEN")
    print("Using HF_TOKEN from Colab secrets.")
except Exception:
    HF_TOKEN = getpass.getpass("Enter your HuggingFace token: ")

os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["HUGGING_FACE_HUB_TOKEN"] = HF_TOKEN
print(f"Token set ({HF_TOKEN[:8]}...)")

## 3. Install Dependencies

Installs Node.js, UV (Python package manager), vLLM, and cloudflared (for the public tunnel).

In [ ]:
%%bash
set -e

echo "=== Installing Node.js ==="
if ! command -v node &>/dev/null || [ "$(node --version | grep -oP '(?<=v)\d+')" -lt 18 ]; then
    curl -fsSL https://deb.nodesource.com/setup_20.x | bash - > /dev/null 2>&1
    apt-get install -y -qq nodejs > /dev/null 2>&1
fi
echo "Node.js $(node --version)"

echo "=== Installing UV ==="
if ! command -v uv &>/dev/null; then
    curl -LsSf https://astral.sh/uv/install.sh | sh 2>/dev/null
fi
export PATH="$HOME/.local/bin:$PATH"
echo "UV $(uv --version)"

echo "=== Installing cloudflared ==="
if ! command -v cloudflared &>/dev/null; then
    curl -fsSL https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o /usr/local/bin/cloudflared
    chmod +x /usr/local/bin/cloudflared
fi
echo "cloudflared $(cloudflared --version 2>&1 | head -1)"

echo "=== Done ==="

In [ ]:
# Install vLLM + HuggingFace CLI (takes a few minutes)
!pip install -q vllm huggingface_hub
!huggingface-cli login --token $HF_TOKEN 2>/dev/null
print("vLLM + HuggingFace ready")

## 4. Clone & Build

In [ ]:
%%bash
set -e
export PATH="$HOME/.local/bin:$PATH"
WORKDIR="/content/docgemma"

echo "=== Cloning repositories ==="
if [ ! -d "$WORKDIR/docgemma-connect" ]; then
    git clone --depth 1 https://github.com/galinilin/docgemma-connect.git "$WORKDIR/docgemma-connect"
else
    echo "docgemma-connect already cloned"
fi

if [ ! -d "$WORKDIR/docgemma-frontend" ]; then
    git clone --depth 1 https://github.com/galinilin/docgemma-frontend.git "$WORKDIR/docgemma-frontend"
else
    echo "docgemma-frontend already cloned"
fi

echo "=== Installing backend dependencies ==="
cd "$WORKDIR/docgemma-connect"
uv sync --frozen --no-dev

echo "=== Building frontend ==="
cd "$WORKDIR/docgemma-frontend"
npm install --silent 2>/dev/null
VITE_API_URL=/api npm run build

echo "=== Copying frontend into backend ==="
mkdir -p "$WORKDIR/docgemma-connect/static"
cp -r "$WORKDIR/docgemma-frontend/dist/"* "$WORKDIR/docgemma-connect/static/"

echo "=== Build complete ==="

## 5. Start vLLM

Starts the vLLM inference server in the background and waits for it to be ready. First run downloads model weights.

In [ ]:
import subprocess, time, urllib.request

# Kill any existing vLLM process
subprocess.run(["pkill", "-f", "vllm.entrypoints"], capture_output=True)
time.sleep(2)

vllm_cmd = [
    "vllm", "serve", MODEL,
    "--max-model-len", "8192",
    "--gpu-memory-utilization", "0.90",
    "--host", "0.0.0.0",
    "--port", str(VLLM_PORT),
]

vllm_log = open("/content/vllm.log", "w")
vllm_proc = subprocess.Popen(vllm_cmd, stdout=vllm_log, stderr=subprocess.STDOUT)
print(f"vLLM starting (PID: {vllm_proc.pid})...")
print(f"Model: {MODEL}")
print("Waiting for model to load (check /content/vllm.log for progress)...")

# Wait for health endpoint
for i in range(360):  # 30 minutes max
    try:
        urllib.request.urlopen(f"http://localhost:{VLLM_PORT}/health", timeout=2)
        print(f"\nvLLM is ready! (took ~{i * 5}s)")
        break
    except Exception:
        if vllm_proc.poll() is not None:
            print("\nvLLM process died. Last 30 lines of log:")
            !tail -30 /content/vllm.log
            raise RuntimeError("vLLM failed to start.")
        if i % 12 == 0 and i > 0:
            print(f"  Still loading... ({i * 5}s elapsed)")
        time.sleep(5)
else:
    raise RuntimeError("vLLM timed out after 30 minutes. Check: !cat /content/vllm.log")

## 6. Start DocGemma

In [ ]:
import subprocess, time, urllib.request, os

# Kill any existing backend process
subprocess.run(["pkill", "-f", "docgemma.api.main"], capture_output=True)
time.sleep(2)

env = os.environ.copy()
env.update({
    "DOCGEMMA_ENDPOINT": f"http://localhost:{VLLM_PORT}",
    "DOCGEMMA_API_KEY": "token",
    "DOCGEMMA_MODEL": MODEL,
    "PATH": f"{os.path.expanduser('~')}/.local/bin:{env.get('PATH', '')}",
})

app_log = open("/content/docgemma.log", "w")
app_proc = subprocess.Popen(
    ["uv", "run", "uvicorn", "docgemma.api.main:app",
     "--host", "0.0.0.0", "--port", str(APP_PORT)],
    cwd=f"{WORKDIR}/docgemma-connect",
    stdout=app_log, stderr=subprocess.STDOUT,
    env=env,
)
print(f"DocGemma starting (PID: {app_proc.pid})...")

# Wait for health
for i in range(30):
    try:
        urllib.request.urlopen(f"http://localhost:{APP_PORT}/api/health", timeout=2)
        print(f"DocGemma is ready on port {APP_PORT}!")
        break
    except Exception:
        if app_proc.poll() is not None:
            print("DocGemma failed to start. Log:")
            !tail -20 /content/docgemma.log
            raise RuntimeError("DocGemma failed to start.")
        time.sleep(2)
else:
    raise RuntimeError("DocGemma timed out. Check: !cat /content/docgemma.log")

## 7. Create Public URL

Creates a Cloudflare tunnel to expose DocGemma via a public URL. No signup required.

In [ ]:
import subprocess, time, re

# Kill any existing tunnel
subprocess.run(["pkill", "-f", "cloudflared"], capture_output=True)
time.sleep(1)

tunnel_log = open("/content/tunnel.log", "w")
tunnel_proc = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", f"http://localhost:{APP_PORT}",
     "--no-autoupdate"],
    stdout=tunnel_log, stderr=subprocess.STDOUT,
)

# Wait for tunnel URL to appear in logs
public_url = None
for i in range(30):
    time.sleep(2)
    try:
        with open("/content/tunnel.log", "r") as f:
            log_content = f.read()
        match = re.search(r"(https://[a-z0-9-]+\.trycloudflare\.com)", log_content)
        if match:
            public_url = match.group(1)
            break
    except Exception:
        pass

if public_url:
    print("")
    print("=" * 60)
    print(f"  DocGemma is live at:")
    print(f"  {public_url}")
    print("=" * 60)
    print("")
    print(f"Model: {MODEL}")
    print(f"GPU:   {gpu_name}")
    print("")
    print("The URL stays active as long as this notebook is running.")
    print("To stop: Runtime > Disconnect and delete runtime")
else:
    print("Failed to create tunnel. Log:")
    !cat /content/tunnel.log
    print(f"\nYou can still access DocGemma locally at: http://localhost:{APP_PORT}")

## Debugging

Run these cells if something goes wrong.

In [ ]:
# Check vLLM logs
!tail -30 /content/vllm.log

In [ ]:
# Check DocGemma logs
!tail -30 /content/docgemma.log

In [ ]:
# Check tunnel logs
!tail -30 /content/tunnel.log

In [ ]:
# Check all processes are running
!ps aux | grep -E 'vllm|docgemma|cloudflared' | grep -v grep

In [ ]:
# Test health endpoints
import urllib.request, json
try:
    resp = urllib.request.urlopen(f"http://localhost:{VLLM_PORT}/health")
    print(f"vLLM:     OK ({resp.status})")
except Exception as e:
    print(f"vLLM:     FAILED ({e})")

try:
    resp = urllib.request.urlopen(f"http://localhost:{APP_PORT}/api/health")
    data = json.loads(resp.read())
    print(f"DocGemma: OK ({data})")
except Exception as e:
    print(f"DocGemma: FAILED ({e})")